# Hadoop Streaming assignment 1: Words Rating

The purpose of this task is to create your own WordCount program for Wikipedia dump processing and learn basic concepts of the MapReduce.

In this task you have to find the 7th word by popularity and its quantity in the reverse order (most popular first) in Wikipedia data (`/data/wiki/en_articles_part`).

There are several points for this task:

1) As an output, you have to get the 7th word and its quantity separated by a tab character.

2) You must use the second job to obtain a totally ordered result.

3) Do not forget to redirect all trash and output to /dev/null.

Here you can find the draft of the task main steps. You can use other methods for solution obtaining.

## Step 1. Create mapper and reducer.

<b>Hint:</b>  Demo task contains almost all the necessary pieces to complete this assignment. You may use the demo to implement the first MapReduce Job.

In [1]:
%%writefile mapper1.py

# Your code for mapper here.
import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper1.py


In [9]:
%%writefile reducer1.py

# Your code for reducer here.
import sys

current_word = None
current_count = 0
word = None

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
        if current_word == word:
            current_count += count
        else:
            if current_word:
                print '%s\t%s' % (current_word, current_count)
            current_count = count
            current_word = word            
    except ValueError as e:
        continue

if current_word == word:
    print '%s\t%s' % (current_word, current_count)       

Overwriting reducer1.py


In [3]:
# You can use this cell for other experiments: for example, for combiner.

## Step 2. Create sort job.

<b>Hint:</b> You may use MapReduce comparator to solve this step. Make sure that the keys are sorted in ascending order.

In [1]:
%%writefile mapper2.py

import sys

current_word = None
current_count = 0
word = None

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
        print '%s\t%s' % (current_word, current_count)
         
    except ValueError as e:
        continue


Overwriting mapper2.py


In [2]:
%%writefile reducer2.py

import sys

current_word = None
current_count = 0
word = None

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
        print '%s\t%s' % (current_word, current_count)
         
    except ValueError as e:
        continue

Overwriting reducer2.py


## Step 3. Bash commands

<b> Hint: </b> For printing the exact row you may use basic UNIX commands. For instance, sed/head/tail/... (if you know other commands, you can use them).

To run both jobs, you must use two consecutive yarn-commands. Remember that the input for the second job is the ouput for the first job.

In [ ]:
%%bash

OUT_DIR="assignment1_"$(date +"%s%6N")

# Code for your first job
NUM_REDUCERS=8
hdfs dfs -rm -r -skipTrash ${OUT_DIR}* > /dev/null
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null
cat ${OUT_DIR}
# Code for your second job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...


In [ ]:
NUM_REDUCERS_2=1
OUT_DIR_2="assignment1_2"$(date +"%s%6N")

hdfs dfs -rm -r -skipTrash ${OUT_DIR_2}* > /dev/null
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount 2" \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D map.output.key.field.separator="\t" \
    -D mapreduce.partition.keycomparator.options="-k1,2nr" \
    -D mapreduce.job.reduces=${NUM_REDUCERS_2} \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python reducer2.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR_2} > /dev/null
# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR_2}/part-00000 | sed -n '7p;8q'
hdfs dfs -rm -r -skipTrash ${OUT_DIR}* > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR_2}* > /dev/null

In [24]:
%%bash
OUT_DIR="assignment1_"$(date +"%s%6N")
hdfs dfs -cat ${OUT_DIR}_2/part-00000 | sed -n '7p;8q'


cat: `assignment1_1565583767853452_2/part-00000': No such file or directory


In [3]:
%%bash
hdfs dfs -rm -r -skipTrash ${OUT_DIR}* > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR_2}* > /dev/null

rm: `comparator1.py': No such file or directory
rm: `mapper1.py': No such file or directory
rm: `mapper2.py': No such file or directory
rm: `mapper.py': No such file or directory
rm: `README.md': No such file or directory
rm: `reducer1.py': No such file or directory
rm: `reducer2.py': No such file or directory
rm: `reducer.py': No such file or directory
rm: `StopWordsTask2.ipynb': No such file or directory
rm: `supervisord.log': No such file or directory
rm: `supervisord.pid': No such file or directory
rm: `WordCountTask0.ipynb': No such file or directory
rm: `WordsRatingTask1b.ipynb': No such file or directory
rm: `WordsRatingTask1.ipynb': No such file or directory
rm: `comparator1.py': No such file or directory
rm: `mapper1.py': No such file or directory
rm: `mapper2.py': No such file or directory
rm: `mapper.py': No such file or directory
rm: `README.md': No such file or directory
rm: `reducer1.py': No such file or directory
rm: `reducer2.py': No such file or directory
rm: `reducer.